# Modeling : KNN

* ## Load the data and analysis libraries

> ### Load libraries

In [1]:
from  datetime import datetime, timedelta
# import gc
import numpy as np, pandas as pd
from sklearn.neighbors import KNeighborsRegressor
import time	
from sklearn.externals import joblib

/Users/Tommy/opt/anaconda3/envs/python36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


> ### We are using the local test from now on.

In [4]:
IS_LOCAL_DEMO = True
# IS_LOCAL_DEMO = False

In [13]:
max_lags = 57
last_day = 1913  
fday = datetime(2016,4, 25) 

> ### load processed data 

In [14]:
data_file = ("train_data.csv.gz" if(IS_LOCAL_DEMO) else "train_data.csv")
dt = pd.read_csv(data_file)

print("loading:",data_file)

loading: train_data.csv.gz


In [15]:
dt["date"] = pd.to_datetime(dt["date"])  


> ### We are using the deep copy to save the data temporarily for the sliding window model to reuse it.

In [16]:
df = dt.copy()

> ### To avoid the model we constructed overfitting, we did not use the previous day to predict rather than created more means as new features. The 7_28 refers to the average of the first 7 days of the last cycle (7), that is, log = 7, wins = 28.

In [17]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):  # [(7, lag_7), (28, lag_28)]
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)  

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean()) 

    date_features = {"wday": "weekday", "week": "weekofyear", "month": "month","quarter": "quarter",  "year": "year","mday": "day" }
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [18]:
create_fea(df)

In [21]:
df.columns

Index(['id', 'item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_7', 'lag_28',
       'rmean_7_7', 'rmean_28_7', 'rmean_7_28', 'rmean_28_28', 'week',
       'quarter', 'mday'],
      dtype='object')

In [26]:
df.dropna(inplace = True)
df.shape

(90000, 31)

>### Split the dataset to training and testing. 

In [27]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday", 'event_name_1', 'event_name_2']
train_cols = df.columns[~df.columns.isin(useless_cols)]
# print(train_cols)
X_train = df[train_cols]
y_train = df["sales"]

In [28]:
X_train.head(3)

,item_id,dept_id,store_id,cat_id,state_id,wday,month,year,event_type_1,event_type_2,...,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
550000,0,0,0,0,0,1,4,2016,0,0,...,8.38,1.0,1.0,1.571429,1.000000,1.071429,1.035714,15,2,16
550001,0,0,0,0,0,2,4,2016,0,0,...,8.38,2.0,3.0,1.714286,1.428571,1.142857,1.142857,15,2,17
550002,0,0,0,0,0,3,4,2016,0,0,...,8.38,0.0,1.0,1.714286,1.571429,1.142857,1.035714,16,2,18


> ### Create a KNN Regressor model to training all selected features by Skit-learn.

In [29]:
model_knn = KNeighborsRegressor(3)
model_knn.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                    weights='uniform')

> ### Unsupervised learning can easily overfit the model. We try to use different weights to obtain the final predicted value to avoid overfitting. After multiple sets of random weight tests, the current optimal weight parameter is[1.035, 1.03, 1.025] on Kaggle.  Model optimization is highly dependent on weight hyperparameters. We need to use more random weight parameters to be optimized the model later.

In [32]:
if(IS_LOCAL_DEMO):
    alphas = [1] # for local demo
else:
    alphas = [1.035, 1.03, 1.025] #[1.015,1.25,1.03] #[1.0,1.15,1.01] #[1.0,1.15,1.01] # for kaggle

weights = [1 / len(alphas)] * len(alphas) 
print(weights)

[1.0]


> ### Build a loop for the sliding window model.

In [31]:
sub = 0.
cols = [f"F{i}" for i in range(1,29)]

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):  # [(1.035, 0.333), (1.03, 0.333), (1.025, 0.333)]
    te = dt # get the all dataset
    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)  # single predict day
        print(icount, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy() # get sliding window date of dataset  
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]  # sliding window select data
        te.loc[te.date == day, "sales"] = alpha * model_knn.predict(tst) # train sliding window select data
    
    # formate
    te_sub = te.loc[te.date >= fday, ["id", "sales"]]  
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]  # [F1,F2,F3,...F28]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()  # change the formate same with the submission
    te_sub.fillna(0., inplace = True) 
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    
     # Calculation weighting
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight 
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)



0 2016-04-25 00:00:00
0 2016-04-26 00:00:00
0 2016-04-27 00:00:00
0 2016-04-28 00:00:00
0 2016-04-29 00:00:00
0 2016-04-30 00:00:00
0 2016-05-01 00:00:00
0 2016-05-02 00:00:00
0 2016-05-03 00:00:00
0 2016-05-04 00:00:00
0 2016-05-05 00:00:00
0 2016-05-06 00:00:00
0 2016-05-07 00:00:00
0 2016-05-08 00:00:00
0 2016-05-09 00:00:00
0 2016-05-10 00:00:00
0 2016-05-11 00:00:00
0 2016-05-12 00:00:00
0 2016-05-13 00:00:00
0 2016-05-14 00:00:00
0 2016-05-15 00:00:00
0 2016-05-16 00:00:00
0 2016-05-17 00:00:00
0 2016-05-18 00:00:00
0 2016-05-19 00:00:00
0 2016-05-20 00:00:00
0 2016-05-21 00:00:00
0 2016-05-22 00:00:00
0 1 1.0


> ### Create validation and evaluation the same with sample submission format.

In [ ]:
sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)

In [ ]:
print(sub.shape)
print(sub.columns)
sub.head()

* ## Dataset persistence
>###  Save the data frame to the hard drive and use the "4agg-12lelvel" script to generate and upload it to The Kaggle for final score evaluation.


In [ ]:
if(IS_LOCAL_DEMO_LOCAL_DEMO):
    train_data.to_csv("submission_knn.csv.gz",compression='gzip',index=False)# for local demo
else:
    train_data.to_csv("submission_knn.csv",index=False) # for kaggle  
